# Calculator Applet

Demo that does various calculations - sha, hmac, pbkdf, field math, elliptic curve math

In [1]:
from applets.helpers import get_connection, AppletBase

class Calculator(AppletBase):
    def __init__(self, connection=None):
        super().__init__("B00B5111CC01", connection)

    def sha256(self, d):
        data = bytes([len(d)])+d
        r = "B0A10000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha256(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A20000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def sha512(self, d):
        data = bytes([len(d)])+d
        r = "B0A30000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)
    
    def hmac_sha512(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A40000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

    def pbkdf2(self, k, d):
        data = bytes([len(k)])+k+bytes([len(d)])+d
        r = "B0A50000"+(bytes([len(data)])+data).hex().upper()
        return self.request(r)

In [2]:
connection = get_connection()
if connection is not None:
    atr=bytes(connection.getATR())
    print(atr.hex())
else:
    print("Failed to open a connection. No cardreaders?")

3bdc18ff8191fe1fc38073c821136605036351000250


In [3]:
app = Calculator(connection)
app.select()

b''

In [4]:
d = b'hello'
k = b'key'

In [6]:
# sha functions from the card
print("sha256:\t\t", app.sha256(d).hex())
print("hmac-sha256:\t", app.hmac_sha256(k, d).hex())
print("sha512:\t\t", app.sha512(d).hex())
print("hmac-sha512:\t", app.hmac_sha512(k, d).hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92


In [7]:
print("pbkdf2:\t\t", app.pbkdf2(k, d).hex())

pbkdf2:		 d881087d013366ea8ad01e704792a7d937f95b1c95cb7cd791b1912b5e7abcf1d08c77877bc5945179c13ac3e15c49fb60492ba5c74c2dd15ad01b11a4ffe660


In [8]:
import hashlib, hmac
def pbkdf2_hmac_sha512(password, salt, iterations:int, bytes_to_read:int):
    # xor two arrays
    def binxor(a, b):
        return bytes([x ^ y for (x, y) in zip(a, b)])
    # convert to bytes
    if isinstance(password, str):
        password = password.encode('utf-8')
    if isinstance(salt, str):
        salt = salt.encode('utf-8')
    # result
    r = b''
    # no need in the loop if bytes_to_read <= 64
    # just set i = 1
    for i in range(1,bytes_to_read//64+1+int(bool(bytes_to_read%64))):
        U = hmac.new(password, salt + i.to_bytes(4,'big'), digestmod=hashlib.sha512).digest()
        result = U
        for j in range(2, 1+iterations):
            U = hmac.new(password, U, digestmod=hashlib.sha512).digest()
            result = binxor(result, U)
        r += result
    return r[:bytes_to_read]

In [13]:
# same using hashlib
print("sha256:\t\t",hashlib.sha256(d).digest().hex())
print("hmac-sha256:\t",hmac.new(k, d, digestmod=hashlib.sha256).digest().hex())
print("sha512:\t\t",hashlib.sha512(d).digest().hex())
print("hmac-sha512:\t",hmac.new(k, d, digestmod=hashlib.sha512).digest().hex())
print("pbkdf2:\t\t", pbkdf2_hmac_sha512(k, d, 2048, 64).hex())

sha256:		 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hmac-sha256:	 9307b3b915efb5171ff14d8cb55fbcc798c6c0ef1456d66ded1a6aa723a58b7b
sha512:		 9b71d224bd62f3785d96d46ad3ea3d73319bfbc2890caadae2dff72519673ca72323c3d99ba5c11d7c7acc6e14b8c5da0c4663475c2e5c3adef46f73bcdec043
hmac-sha512:	 ff06ab36757777815c008d32c8e14a705b4e7bf310351a06a23b612dc4c7433e7757d20525a5593b71020ea2ee162d2311b247e9855862b270122419652c0c92
pbkdf2:		 d3690a2849eea8253f1f85ec4ed890a20e5cecc94f489d259e5c2dbbce931c8bbab004739416ebf5122e6889365724642274c408d45681a95693404ade8e6de8


In [ ]:
connection.disconnect()